# Поиск по коллекции (из фактов Википедии)
Шулюгин Иван МГУ ВМК 425  
октябрь 2020  

## Описание TODO
- Даты, римские цифры, английские названия - тоже термы (это может играть роль при поиске документа)
- Есть возможность сохранять или пересобирать коллекцию с помощью make_collection()
- Добавлена автоматическое добавление новых текстов в коллекцию (нужно записывать новые факты в виде fact_*\<number>*.txt в директорию text и перезапустить сборку коллекции)

In [1]:
import numpy as np

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

from pymystem3 import Mystem

import re

import os

import pickle

Для обработки русских предложений, может понадобиться установить сначала токенайзер русского языка.

```
import nltk
nltk.download('punkt')
```

## Задача

Запросы – это проанализированные вами факты из Википедии  

• Коллекция собирается из всех упомянутых статьи из всех фактов  
• Документы – это предложения из статей Википедии, указанных в этих фактах, т.е. коллекция – это объединенная коллекция предложений статей всех фактов  

• Все должно быть обработано морфологическим анализатором  

• Нужно найти наиболее релевантные предложения  
– По tf.idf (df в данном случае – это количество предложений, в которых встречалось слово)  
– Tf –  
    • 1) это количество упоминаний слова в предложении (count) или  
    • 2) log (1+count)  
– Нормализация запроса и предложения  
– Т.е. выстроить все предложения из статей по мере сходства с запросом по векторной модели.  
– В отчете должны быть показаны веса выдаваемых предложений  

## Запросы

1) Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына  
2) К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции  
3) Лагерь сапёров мог стать важнейшим городом Британской Колумбии  

## Решение

### Разбиение текста на предложения

Собираем текст из всех фактов (тексты записаны в файлах **fact_*i*.txt** в исходной директории)

In [2]:
def collect_text():
    reg_file = r'fact_\d.txt'

    file_list = []

    for file_name in sorted(os.listdir('text')):
        if re.match(reg_file, file_name):
            file_list.append('text/'+file_name)

    if file_list == []:
        print("there are no text files")

    all_text = ""

    for file_name in file_list:
        with open(file_name) as file:
            print("open", file_name)
            all_text = all_text + file.read()
    print("done!")

    return all_text

In [3]:
all_text = collect_text()

open text/fact_1.txt
open text/fact_2.txt
open text/fact_3.txt
done!


Разбиваем текст на предложения с помощью токенайзера

In [4]:
def sentence_list(text):
    proc_text = []

    for el in text.split('\n'):
        if el:
            sent_list = sent_tokenize(el, language="russian")
            for s in sent_list:
                proc_text.append(s)
    return proc_text

In [5]:
proc_text = sentence_list(all_text)

In [6]:
proc_text

['Генрих III (1 октября 1207, Уинчестер — 16 ноября 1272, Вестминстер) — король Англии (1216—1272) и герцог Аквитании из династии Плантагенетов, один из самых малоизвестных британских монархов, при том что правил он дольше всех прочих средневековых королей Англии — 56 лет.',
 'Ранние годы.',
 'Генрих родился 1 октября 1207 года в Уинчестерском замке.',
 'Он был старшим сыном короля Иоанна Безземельного и Изабеллы Ангулемской.',
 'О ранней жизни Генриха известно немного.',
 'Первоначально он находился под присмотром кормилицы по имени Элен на юге Англии, вдали от странствующего двора Иоанна, и, вероятно, был близок к своей матери.',
 'У Генриха были четыре законных брата и сестры: Ричард, Иоанн, Изабелла и Элеонора и множество старших незаконнорождённых братьев и сестёр.',
 'В 1212 году его образование было доверено винчестерскому епископу Пьеру де Рошу; под его руководством Генрих получил военную подготовку у Филиппа д’Обиньи и обучился верховой езде (возможно, у Ральфа из св. Самсона)

### Сбор термов, подсчет idf

Регулярное выражение и функция лемматизации:

In [7]:
# кроме русских слов оставим еще даты, а также английские названия,
#   тем самым оставив римские цифры (e.g. III = 3, IV = 4)
reg_filter = r'[а-яА-Я]|[a-zA-Z]|\d'

In [8]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")

def process_text(text):    
    tokens = mystem.lemmatize(text)
    tokens = [token.lower() 
              for token in tokens 
              if token not in russian_stopwords 
              and token not in english_stopwords
              and token != " "
              and re.match(reg_filter, token) ]
    return tokens

Теперь лемматизируем каждое предложение из **proc_text**, удаляем стоп-слова, проверяем на соответствие регулярному выражению заносим уже термы в словарь термов

In [9]:
terms = {}
term_text = []

for sentence in proc_text:
    new_terms = process_text(sentence)
    term_text.append(new_terms)
    
    for t in new_terms:
        if t not in terms:
            terms[t] = {'df': None, 'idf': None}

Определяем df для каждого терма

In [10]:
for t in terms:
    terms[t]['df'] = 0
    for doc in term_text:
        if t in doc:
            terms[t]['df'] += 1

Считаем idf

In [11]:
number_of_docs = len(proc_text)

In [12]:
for t in terms:
    terms[t]['idf'] = np.log10(number_of_docs / terms[t]['df'])

In [13]:
terms

{'генрих': {'df': 55, 'idf': 1.0348835702459926},
 'iii': {'df': 24, 'idf': 1.3950350180286304},
 '1': {'df': 20, 'idf': 1.4742162640762553},
 'октябрь': {'df': 6, 'idf': 1.9970950093565927},
 '1207': {'df': 2, 'idf': 2.4742162640762553},
 'уинчестер': {'df': 1, 'idf': 2.7752462597402365},
 '16': {'df': 4, 'idf': 2.173186268412274},
 'ноябрь': {'df': 7, 'idf': 1.9301482197259796},
 '1272': {'df': 4, 'idf': 2.173186268412274},
 'вестминстер': {'df': 7, 'idf': 1.9301482197259796},
 'король': {'df': 69, 'idf': 0.9363971690029811},
 'англия': {'df': 26, 'idf': 1.3602729117694186},
 '1216': {'df': 3, 'idf': 2.298125005020574},
 'герцог': {'df': 2, 'idf': 2.4742162640762553},
 'аквитания': {'df': 1, 'idf': 2.7752462597402365},
 'династия': {'df': 1, 'idf': 2.7752462597402365},
 'плантагенет': {'df': 3, 'idf': 2.298125005020574},
 'самый': {'df': 10, 'idf': 1.7752462597402365},
 'малоизвестный': {'df': 1, 'idf': 2.7752462597402365},
 'британский': {'df': 62, 'idf': 0.9828545702419825},
 'мона

То есть здесь, в конечном итоге, функция выглядит так:

In [14]:
def make_terms(proc_text):
    terms = {}
    term_text = []

    for sentence in proc_text:
        new_terms = process_text(sentence)
        term_text.append(new_terms)

        for t in new_terms:
            if t not in terms:
                terms[t] = {'df': None, 'idf': None}
                
    for t in terms:
        terms[t]['df'] = 0
        for doc in term_text:
            if t in doc:
                terms[t]['df'] += 1
    
    number_of_docs = len(proc_text)
    for t in terms:
        terms[t]['idf'] = np.log10(number_of_docs / terms[t]['df'])
    
    return terms, term_text

In [15]:
terms, term_text = make_terms(proc_text)

### Отображение исходного предложения в вектор пространства термов

Функция нормализации вектора:

In [16]:
def normalize(vec):
    norm = np.linalg.norm(vec)
    return vec/norm

Функция представления вектора в пространстве термов:

In [17]:
def tf_vec(doc, terms):
    words = list(terms.keys())
    doc_vec = np.zeros(len(terms.keys()))

    for t in doc:
        if t in words:
            i = words.index(t)
            doc_vec[i] += 1
        else:
            print("WARN: query word {" + t + "} is not in the collection")
        
    return doc_vec

Функция взвешенного вектора документа (выдает два ответа, соответственно двум разным способам учета tf):

In [18]:
def weight_tf_idf_vec(doc, terms):
    words = terms.keys()
    w_vec = np.zeros(len(terms.keys()))
    
    doc_vec_tf1 = tf_vec(doc, terms)
    doc_vec_tf2 = tf_vec(doc, terms)
    #print("doc = ", doc, "\ndoc_vec after tf_vec():\n", doc_vec_tf1, "\n")
    
    i = 0
    for word in words:
        doc_vec_tf1[i] *= terms[word]['idf']
        doc_vec_tf2[i] = np.log(1+doc_vec_tf2[i]) * terms[word]['idf']
        i += 1
    
    return (doc_vec_tf1, doc_vec_tf2)

На этом моменте у нас есть два списка: исходные предложения **proc_text** и списки термов каждого предложения **term_text**  
  
Для каждого документа из **proc_text** построим векторы по его представлению в **term_text** и запишем их вместе  

In [19]:
proc_collection = list(zip(proc_text, [weight_tf_idf_vec(sent, terms) for sent in term_text]))

Осталось теперь найти релевантные документы из данной коллекции для запроса

Функция сборки коллекции (сохраняет объект pickle):

In [20]:
def make_collection():
    print("building collection...")
    all_text = collect_text()
    proc_text = sentence_list(all_text)
    terms, term_text = make_terms(proc_text)
    proc_collection = list(zip(proc_text, [weight_tf_idf_vec(sent, terms) for sent in term_text]))
    if 'obj' not in os.listdir():
        os.mkdir('obj')
    with open('obj/core_collection.pkl','wb') as f:
        pickle.dump(proc_collection, f, pickle.HIGHEST_PROTOCOL)
    with open('obj/terms.pkl','wb') as f:
        pickle.dump(terms, f, pickle.HIGHEST_PROTOCOL)

Функция подсчета близости документов:

In [21]:
def proximity(vec1, vec2):
    cos = np.dot(normalize(vec1), normalize(vec2))
    return cos

**Функция поиска по коллекции (выдает документы со значениями по мере их релевантности):**

In [22]:
def search(query):
    obj_name = 'core_collection.pkl'
    if 'obj' not in os.listdir() or obj_name not in os.listdir('obj'):
        make_collection()

    with open('obj/'+obj_name,'rb') as f:
        proc_collection = pickle.load(f)
    with open('obj/terms.pkl','rb') as f:
        terms = pickle.load(f)

    vec_q = normalize(tf_vec(process_text(query), terms))

    rel_docs1 = []
    rel_docs2 = []
    
    for i in range(len(proc_collection)):
        vec_d1 = proc_collection[i][1][0] # вектор по подсчету tf = count
        vec_d2 = proc_collection[i][1][1] # вектор по подсчету tf = log(1+count)
        
        rel_docs1.append((proc_collection[i][0], proximity(vec_q, vec_d1)))
        rel_docs2.append((proc_collection[i][0], proximity(vec_q, vec_d2)))
                         
    rel_docs1.sort(key=lambda x:x[1], reverse=True)
    rel_docs2.sort(key=lambda x:x[1], reverse=True)
        
    return rel_docs1, rel_docs2

### Поиск по запросу

1) Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына  

In [23]:
search('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына')[0][:3]

WARN: query word {награда} is not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  0.3497588024421523),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  0.3080062523852433),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  0.2687454450045788)]

In [24]:
search('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына')[1][:3]

WARN: query word {награда} is not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  0.3497588024421523),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  0.3080062523852433),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  0.2545444967241659)]

2) К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции  

In [25]:
search('К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции')[0][:3]

WARN: query word {хх} is not in the collection
WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection


[('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  0.2582722325002451),
 ('По инициативе Шувалова Баранчинский завод был реконструирован в чугуноплавильный.',
  0.2577126642065135),
 ('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  0.2504244714330281)]

In [26]:
search('К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции')[1][:3]

WARN: query word {хх} is not in the collection
WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection


[('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  0.25827223250024517),
 ('По инициативе Шувалова Баранчинский завод был реконструирован в чугуноплавильный.',
  0.2577126642065135),
 ('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  0.2504244714330281)]

3) Лагерь сапёров мог стать важнейшим городом Британской Колумбии  

In [27]:
search('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')[0][:5]

[('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  0.2622196898521864),
 ('Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.',
  0.23909099746446222),
 ('Туризм также стали играть важную роль в экономике.', 0.22778274362385423),
 ('Королева Виктория выбрала название Британская Колумбия, чтобы отличать её от округа Колумбия в Соединённых Штатах («Американская Колумбия» или «Южная Колумбия»), которая стала территорией штата Орегон в 1848 году в результате договора.',
  0.22705114711976299),
 ('Название произошло от района Колумбии, британского названия территории по реке Колумбия, которая брала свои истоки на юго-востоке Британской Колумбии.',
  0.22602370841159297)]

In [28]:
search('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')[1][:5]

[('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  0.2423808893362787),
 ('Туризм также стали играть важную роль в экономике.', 0.2277827436238542),
 ('Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.',
  0.21712823838191134),
 ('Только с согласия этого органа король мог назначать министров и принимать важные решения по управлению страной.',
  0.21299562248894288),
 ('Столица Британской Колумбии — город Виктория — расположена в юго-восточной оконечности острова Ванкувер.',
  0.20645422589527457)]

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---

In [29]:
test = process_text('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')

In [30]:
test_vec = tf_vec(test, terms)
i=0
for x in test_vec:
    if x:
        print(list(terms.keys())[i])
    i+=1

британский
становиться
город
важный
мочь
колумбия
лагерь
сапер


In [31]:
test_doc = process_text('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')

vec1, vec2 = weight_tf_idf_vec(test_doc, terms)

In [32]:
proximity(test_vec, vec1)

0.9487837494964227

In [33]:
proximity(test_vec, vec2)

0.9487837494964226

In [34]:
proc_collection[0][1][0] # вектор по подсчету tf = count

array([1.03488357, 1.39503502, 1.47421626, ..., 0.        , 0.        ,
       0.        ])

In [35]:
proc_collection[0][1][1] # вектор по подсчету tf = log(1+count)

array([0.71732663, 0.96696459, 1.02184885, ..., 0.        , 0.        ,
       0.        ])

In [36]:
proximity(test_vec, proc_collection[0][1][0])

0.02802355880610595

In [37]:
proximity(test_vec, proc_collection[0][1][1])

0.02909180154377329

In [38]:
for v in weight_tf_idf_vec(test_doc, terms):
    for val in v:
        print(val, end=' ')
    print('\n\n\n\n\n\n\n!!!!!!!')

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.9828545702419825 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.2311782153899609 0.0 0.0 0.0 0.0 0.0 0.0 0.

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.6812628742636869 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.8533877087643764 0.0 0.0 0.0 0.0 0.0 0.0 0.

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0